In [ ]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
def scrape_all():
    # Initiate headless driver for deployment
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)


In [ ]:
news_title, news_paragraph = mars_news(browser)
hemisphere_image_urls=hemisphere(browser)


In [ ]:
# Run all scraping functions and store results in dictionary
data = {
      "news_title": news_title,
      "news_paragraph": news_paragraph,
      "featured_image": featured_image(browser),
      "facts": mars_facts(),
      "last_modified": dt.datetime.now()
}

In [ ]:
# Stop webdriver and return data
   browser.quit()
   return data

def mars_news(browser):
    

In [ ]:
# Scrape Mars News
    # Visit the mars nasa news site
    url = 'https://data-class-mars.s3.amazonaws.com/Mars/index.html'
    browser.visit(url)


In [ ]:
# Optional delay for loading the page
    browser.is_element_present_by_css('div.list_text', wait_time=1)


In [ ]:
# Convert the browser html to a soup object and then quit the browser
    html = browser.html
    news_soup = soup(html, 'html.parser')


In [ ]:
# Add try/except for error handling
    try:
        slide_elem = news_soup.select_one('div.list_text')
        # Use the parent element to find the first 'a' tag and save it as 'news_title'
        news_title = slide_elem.find('div', class_='content_title').get_text()
        # Use the parent element to find the paragraph text
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

    except AttributeError:
        return None, None

    return news_title, news_p


def featured_image(browser):

In [ ]:
# Visit URL
    url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    browser.visit(url)


In [ ]:
Find and click the full image button
    full_image_elem = browser.find_by_tag('button')[1]
    full_image_elem.click()


In [ ]:
# Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')


In [ ]:
# Add try/except for error handling
    try:
        # Find the relative image url
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
        
    except AttributeError:
        return None

In [ ]:
# Use the base url to create an absolute url
    img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
    print(img_url)
    return img_url

def mars_facts():

In [ ]:
 # Add try/except for error handling
    try:
        # Use 'read_html' to scrape the facts table into a dataframe
        df = pd.read_html('https://data-class-mars-facts.s3.amazonaws.com/Mars_Facts/index.html')[0]

    except BaseException:
        return None


In [ ]:
# Assign columns and set index of dataframe
    df.columns=['Description', 'Mars', 'Earth']
    df.set_index('Description', inplace=True)


In [ ]:
# Convert dataframe into HTML format, add bootstrap
    return df.to_html(classes="table table-striped")

def hemisphere(browser):
    url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)


    hemisphere_image_urls = []

    imgs_links= browser.find_by_css("a.product-item h3")

    for x in range(len(imgs_links)):
        hemisphere={}

In [ ]:
# Find elements going to click link 
        browser.find_by_css("a.product-item h3")[x].click()


In [ ]:
# Find sample Image link
        sample_img= browser.find_link_by_text("Sample").first
        hemisphere['img_url']=sample_img['href']


In [ ]:
# Get hemisphere Title
hemisphere['title']=browser.find_by_css("h2.title").text


In [ ]:
#Add Objects to hemisphere_img_urls list
hemisphere_image_urls.append(hemisphere)


In [ ]:
# If running as script, print scraped data
    print(scrape_all())
